In [1]:
!pip install pandas_ta
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=e9933e142067f25129d7357fe7c0389e4bbc78a2b56502d170ec9ac3f242d09b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import pandas_ta as ta
from pandas import read_csv
from pandas.core.frame import DataFrame
import tensorflow as tf
import os
import numpy as np
from numpy import split, array, dstack
import math
from math import sqrt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Flatten, Dropout
from keras.layers import LSTM , GRU
from keras.optimizers import Adam
import csv
from scipy.stats import zscore
from datetime import datetime
import pickle
from keras.models import load_model
from warnings import simplefilter
from scipy.stats import pearsonr
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [4]:
print(tf.__version__)

2.17.1


## Required functions

### Company data feature engineering functions

In [5]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
  return pd.DataFrame(values).fillna(method='ffill').values


def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#def zlema(src, length):
#    lag = round((length - 1) / 2)
#    ema_data = src + (src - src[lag])
#    zl= ta.ema(ema_data,length = 5)
#    return zl


In [6]:
def get_company_df(company,company_name):
  new_date_range = pd.date_range(start="31-12-2001", end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.NaN)
  company.replace('?', np.NaN, inplace=True)
  company = company.astype('float32')
  company[:] = fill_missing(company.values)
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['ad'] = ta.ad(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'],length=5)
  company['adosc'] = ta.adosc(high=company['High'],low= company['Low'],volume=company['Volume'],close=company['Close'],fast=1,slow=5)
  company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['alma'] = ta.alma(company['Close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['Close'],length=5)
  company['ao'] = ta.ao(high=company['High'],low= company['Low'],close=company['Close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['Close'],company['Volume'])
  company['apo'] = ta.apo(company['Close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['atr']= ta.atr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['Close'],length=5)
  company['bias'] = ta.bias(company['Close'],length=5)
  company['bop']= ta.bop(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cci']= ta.cci(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cfo'] = ta.cfo(company['Close'],length=5)
  company['cg'] = ta.cg(company['Close'],length=5)
  company['chop']= ta.chop(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['High'],low= company['Low'],close=company['Close'])
  company['cmf']= ta.cmf(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'],open = company['Open'],length=5)
  company['cmo'] = ta.cmo(company['Close'],length=5)
  company['coppock'] = ta.coppock(company['Close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['Close'],length=5)
  company['decay'] = ta.decay(company['Close'],length=5)
  company['decreasing'] = ta.decreasing(company['Close'],length=5)
  company['dema'] = ta.dema(company['Close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['High'],low= company['Low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['High'],low= company['Low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['Close'],length=5)
  company['ebsw'] = ta.ebsw(company['Close'],length=5)
  company['efi'] = ta.efi(company['Close'],company['Volume'],length=5)
  company['ema'] = ta.ema(company['Close'],length=5)
  company['entropy'] = ta.entropy(company['Close'],length=5)
  company['eom']= ta.eom(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'],length=5)
  company['er'] = ta.er(company['Close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['High'],low= company['Low'],length=5)
  company['fwma'] = ta.fwma(company['Close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['Close'],high=company['High'],low= company['Low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['Close'],length=5)
  company['hl2'] = ta.hl2(company['High'],company['Low'])
  company['hlc3'] = ta.hlc3(company['High'],company['Low'],company['Close'])
  company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['Close'])
  company['hwma '] = ta.dema(company['Close'])
  company['increasing'] = ta.increasing(company['Close'],length=5)
  company['inertia'] = ta.inertia(high=company['High'],low= company['Low'],close=company['Close'],open = company['Open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['Close'],length=5)
  company['kama'] = ta.kama(company['Close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['Close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['High'],low= company['Low'],close=company['Close'],volume = company['Volume'])
  company['linreg'] = ta.linreg(company['Close'],length=5)
  company['log_return'] = ta.log_return(company['Close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['Close'])
  company['mad'] = ta.mad(company['Close'],length=5)
  company['massi'] = ta.massi(high=company['High'],low= company['Low'],fast=1,slow=5)
  company['median'] = ta.median(company['Close'],length=5)
  company['mfi'] = ta.mfi(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'],length=5)
  company['midpoint'] = ta.midpoint(company['Close'],length=5)
  company['midprice'] = ta.midprice(high=company['High'],low= company['Low'],length=5)
  company['mom'] = ta.mom(company['Close'],length=5)
  company['natr'] = ta.natr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['nvi'] = ta.nvi(company['Close'],company['Volume'],length=5)
  company['obv'] = ta.obv(company['Close'],company['Volume'])
  company['pdist'] = ta.pdist(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'])
  company['percent_return'] = ta.percent_return(company['Close'],length=5)
  company['pgo'] = ta.pgo(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['psl'] = ta.psl(company['Close'],company['Open'],length=5)
  company['pvi'] = ta.pvi(company['Close'],company['Volume'],length=5)
  company['pvol'] = ta.pvol(company['Close'],company['Volume'])
  company['pvt'] = ta.pvt(company['Close'],company['Volume'])
  company['pwma'] = ta.pwma(company['Close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['Close'],length=5)
  company['qstick'] = ta.qstick(company['Open'],company['Close'],length=5)
  company['quantile'] = ta.quantile(company['Close'],length=5)
  company['rma'] = ta.rma(company['Close'],length=5)
  company['roc'] = ta.roc(company['Close'],length=5)
  company['rsi'] = ta.rsi(company['Close'],length=5)
  company['rsx'] = ta.rsx(company['Close'],length=5)
  company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['Open'],high=company['High'],low= company['Low'],close=company['Close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['sinwma'] = ta.sinwma(company['Close'],length=5)
  company['skew'] = ta.skew(company['Close'],length=5)
  company['slope'] = ta.slope(company['Close'],length=5)
  company['sma'] = ta.sma(company['Close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['Close'])
  company['ssf'] = ta.ssf(company['Close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['Close'],tclength=5)
  company['stdev'] = ta.stdev(company['Close'],length=5)
  company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['High'],low= company['Low'],close=company['Close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['Close'],length=5)
  company['swma'] = ta.swma(company['Close'],length=5)
  company['t3'] = ta.t3(company['Close'],length=5)
  company['tema'] = ta.tema(company['Close'],length=5)
  company['trima'] = ta.trima(company['Close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['Close'],length=5)
  company['true_range'] = ta.true_range(high=company['High'],low= company['Low'],close=company['Close'])
  company['tema'] = ta.tema(company['Close'],length=5)
  company['ui'] = ta.ui(high=company['High'],close =company['Close'],length=5)
  company['uo'] = ta.uo(company['High'],company['Low'],company['Close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['Close'],length=5)
  company['vhf'] = ta.vhf(company['Close'],length=5)
  company['vidya'] = ta.vidya(company['Close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['High'],company['Low'],company['Close'],length=5)
  company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['Close'],company['Volume'])
  company['vwap'] = ta.vwap(high=company['High'],low= company['Low'],close=company['Close'],volume=company['Volume'])
  company['vmwa'] = ta.vwma(volume=company['Volume'],close=company['Close'],length=5)
  company['wcp'] = ta.wcp(high=company['High'],low= company['Low'],close=company['Close'])
  company['willr'] = ta.willr(high=company['High'],low= company['Low'],close=company['Close'],length=5)
  company['wma'] = ta.wma(company['Close'],length=5)
  company['zlma'] = ta.zlma(company['Close'],length=5)
  company['zscore'] = ta.zscore(company['Close'],length=5)
  #company['zlema'] = zlema(company['Close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  #selected_ti = ['Open','High','Low','Close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  #new_company_df =  company[selected_ti]
  return company

## Excution

In [7]:
def get_correlated_features(company_name,company_path):
  company = pd.read_csv(company_path,index_col='Date',parse_dates=True)
  company_df = get_company_df(company,company_name)
  company_df.replace([np.inf, -np.inf], 0, inplace=True)
  company_features = []
  df_dict = {technical_indicator: 0.0 for technical_indicator in list(company_df.columns)}
  for technical_indicator in df_dict:
    if company_df[f'{technical_indicator}'].nunique() == 1:
        continue
    corr, _ = pearsonr(company_df['Close'], company_df[f'{technical_indicator}'])
    print(technical_indicator,", ",corr)
    df_dict[technical_indicator] = corr
    if(corr>0.8):
      company_features.append(technical_indicator)
  return company_features

In [8]:
from scipy.stats import pearsonr
import numpy as np
import pandas as pd

def get_correlated_features2(company_name, company_path):
    company = pd.read_csv(company_path, index_col='Date', parse_dates=True)
    company_features = []
    df_dict = {technical_indicator: 0.0 for technical_indicator in list(company.columns)}

    for technical_indicator in df_dict:
        if company[technical_indicator].nunique() == 1:
            # Skip features with constant values
            continue

        # Clean data for valid correlation
        valid_data = company[['Close', technical_indicator]].replace([np.inf, -np.inf], np.nan).dropna()
        if valid_data.empty:
            continue

        # Extract 1D arrays for correlation
        close_series = valid_data['Close'].values.flatten()
        indicator_series = valid_data[technical_indicator].values.flatten()

        # Calculate correlation
        corr, _ = pearsonr(close_series, indicator_series)
        df_dict[technical_indicator] = corr

        # Add features with high correlation
        if corr > 0.8:
            company_features.append(technical_indicator)

    return company_features


In [9]:
aapl = get_correlated_features("AAPL","/content/drive/MyDrive/KFUPM/Graduation/data/AAPL.csv")

<ipython-input-5-ac9c26444dc3>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['Close'],length=5)
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__setitem__ treating keys as positions is deprecate

Low ,  0.999925574252941
Open ,  0.999837956608957
Volume ,  -0.4992794915007216
High ,  0.9999193012818421
Close ,  1.0
Adjusted Close ,  0.9989880902264612
atr ,  0.8418235779294416
ad ,  0.6871897066400934
adosc ,  -0.03892890085541412
alma ,  0.9994805171607879
ao ,  0.09352758049427004
apo ,  0.08879093136800098
bias ,  -0.004347228459064354
bop ,  0.04401128848663549
cci ,  0.04701209773279329
cfo ,  0.0005946370332093835
cg ,  -0.02380756148966235
chop ,  -0.052867567355940163
cmf ,  0.031618668601792736
cmo ,  0.06008839762767184
coppock ,  -0.020590595323662093
cti ,  0.04284496797972212
decay ,  0.999942182303398
decreasing ,  -0.0445318382035097
dema ,  0.9999527046306022
dpo ,  -0.03196236582792551
ebsw ,  0.05850392864474683
efi ,  0.00674460715594301
ema ,  0.9998544378861246
entropy ,  0.03299186270137477
eom ,  0.0472603099284618
er ,  0.017761085827564603
fwma ,  0.9999108914514541
hma ,  0.9999493217896164
hl2 ,  0.9999523448676231
hlc3 ,  0.9999788259249189
hwma  ,  

In [10]:
bac = get_correlated_features("BAC","/content/drive/MyDrive/KFUPM/Graduation/data/BAC.csv")
bac

<ipython-input-5-ac9c26444dc3>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['Close'],length=5)
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__setitem__ treating keys as positions is deprecat

Low ,  0.9997044824905696
Open ,  0.9994645220828103
Volume ,  -0.6385671910756948
High ,  0.9996961647007312
Close ,  1.0
Adjusted Close ,  0.9642645102423876
atr ,  0.296813538404199
ad ,  0.746291754690208
adosc ,  0.06127800821110363
alma ,  0.9960221381655917
ao ,  0.031889685097732254
apo ,  0.03030428056103412
bias ,  0.021481758105351613
bop ,  0.05476479150785202
cci ,  0.040861262612828055
cfo ,  -0.0008224041996500557
cg ,  0.016350033813400517
chop ,  0.1330394446045557
cmf ,  0.0736259711059431
cmo ,  0.04280588935011461
coppock ,  0.011861851816389959
cti ,  0.04284248528982112
decay ,  0.9998042159306864
decreasing ,  -0.038704053593030474
dema ,  0.9983070216487173
dpo ,  0.030752930140311663
ebsw ,  0.048406658891787
efi ,  0.0012523646554738746
ema ,  0.9980117579033138
entropy ,  -0.0016616018139470942
eom ,  0.026400600495372562
er ,  -0.017277771137256884
fwma ,  0.9981785905295641
hma ,  0.9979078265760398
hl2 ,  0.999840867644802
hlc3 ,  0.9999292945152224
hwma  

['Low',
 'Open',
 'High',
 'Close',
 'Adjusted Close',
 'alma',
 'decay',
 'dema',
 'ema',
 'fwma',
 'hma',
 'hl2',
 'hlc3',
 'hwma ',
 'jma',
 'kama',
 'linreg',
 'median',
 'midpoint',
 'midprice',
 'pvi',
 'pwma',
 'quantile',
 'rma',
 'sinwma',
 'sma',
 'ssf',
 'swma',
 't3',
 'tema',
 'trima',
 'vidya',
 'vwap',
 'vmwa',
 'wcp',
 'wma',
 'zlma']

In [11]:
jnj = get_correlated_features("JNJ","/content/drive/MyDrive/KFUPM/Graduation/data/JNJ.csv")

<ipython-input-5-ac9c26444dc3>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['Close'],length=5)
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__setitem__ treating keys as positions is deprecat

Low ,  0.9998187018884621
Open ,  0.9996284157749348
Volume ,  -0.2879565996363225
High ,  0.9997888166777695
Close ,  0.9999999999999998
Adjusted Close ,  0.9962223530588326
atr ,  0.5139194602154621
ad ,  0.7227036401451185
adosc ,  0.06305513295753931
alma ,  0.9972537014701767
ao ,  0.0436691988438744
apo ,  0.041726091456903786
bias ,  0.032181972111762824
bop ,  0.04648129385071612
cci ,  0.04687893028913065
cfo ,  0.0016905539268502914
cg ,  -0.015290849807792654
chop ,  -0.047919270340850514
cmf ,  0.08674842434417065
cmo ,  0.09404595812651592
coppock ,  0.05650497243030216
cti ,  0.057478639198317104
decay ,  0.9998682018096529
decreasing ,  -0.04192601506977335
dema ,  0.9986944401135204
dpo ,  0.011272131173149517
ebsw ,  0.05590000707456288
efi ,  -0.01985021398037505
ema ,  0.9985212482574479
entropy ,  0.031523541068720684
eom ,  0.002084323026744961
er ,  0.019513027079653345
fwma ,  0.9986192408649969
hma ,  0.99839561987783
hl2 ,  0.9998898464954826
hlc3 ,  0.99995105

In [15]:
goog = get_correlated_features("GOOG","/content/drive/MyDrive/KFUPM/Graduation/data/GOOG.csv")

<ipython-input-5-ac9c26444dc3>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return pd.DataFrame(values).fillna(method='ffill').values
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['Close'],length=5)
<ipython-input-6-b84bff4bdc7d>:32: FutureWarning: Series.__setitem__ treating keys as positions is deprecat

Low ,  0.999926494007042
Open ,  0.9998194933708481
Volume ,  -0.29760087459008955
High ,  0.9999087378937949
Close ,  1.0
Adjusted Close ,  1.0
atr ,  0.8128170775217899
ad ,  0.6987228398650069
adosc ,  0.014330523136698312
alma ,  0.9994686131876266
ao ,  0.05818850045845212
apo ,  0.05491677692329197
bias ,  0.008676062956508166
bop ,  0.042040504443417476
cci ,  0.04174066377128269
cfo ,  -0.00042324074037546934
cg ,  -0.47737724620619537
chop ,  0.3868108323211607
cmf ,  0.06468171194083139
cmo ,  0.0804778370385466
coppock ,  0.012231037699438058
cti ,  0.047993166646910154
decay ,  0.9999430495882937
decreasing ,  0.1352974234900119
dema ,  0.9999512779010757
dpo ,  -0.008684048708036272
efi ,  -0.00988044482919809
ema ,  0.9998541360899476
entropy ,  0.47825232524117145
eom ,  0.012640589303774024
er ,  0.2807202835474149
fwma ,  0.9999038408013105
hma ,  0.9999416155538877
hl2 ,  0.9999473554404498
hlc3 ,  0.9999766066842715
hwma  ,  0.9998444356952625
increasing ,  0.2060586

In [16]:
print(f'AAPL: {len(aapl)}\nBAC: {len(bac)}\nJ&J: {len(jnj)} \nGOOG: {len(goog)}')

AAPL: 39
BAC: 37
J&J: 38 
GOOG: 35


In [17]:
common_features = set(aapl).intersection(bac, jnj, goog)
len(common_features)

33

In [18]:
common_features

{'Adjusted Close',
 'Close',
 'High',
 'Low',
 'Open',
 'alma',
 'decay',
 'dema',
 'ema',
 'fwma',
 'hl2',
 'hlc3',
 'hma',
 'hwma ',
 'linreg',
 'median',
 'midpoint',
 'midprice',
 'pvi',
 'pwma',
 'quantile',
 'rma',
 'sinwma',
 'sma',
 'swma',
 't3',
 'tema',
 'trima',
 'vmwa',
 'vwap',
 'wcp',
 'wma',
 'zlma'}

In [19]:
selected_saudi_ti = ['Open','High','Low','Close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
selected_us_ti = ['Adjusted Close',
 'Close',
 'High',
 'Low',
 'Open',
 'alma',
 'decay',
 'dema',
 'ema',
 'fwma',
 'hl2',
 'hlc3',
 'hma',
 'hwma ',
 'linreg',
 'median',
 'midpoint',
 'midprice',
 'pvi',
 'pwma',
 'quantile',
 'rma',
 'sinwma',
 'sma',
 'swma',
 't3',
 'tema',
 'trima',
 'vmwa',
 'vwap',
 'wcp',
 'wma',
 'zlma']

In [20]:
result = all(elem in selected_us_ti for elem in selected_saudi_ti)

In [22]:
result

False

In [21]:
set(selected_us_ti) - set(selected_saudi_ti)

{'Adjusted Close', 'pvi'}

In [23]:
set(selected_saudi_ti) - set(selected_us_ti)

{'jma', 'kama', 'ssf'}